# READING THE DATA

In [ ]:
import os
import tensorflow as tf
import numpy as np
import math
import random
import tensorflow.contrib.keras as keras
import tensorflow.contrib.layers as layers
import pandas as pd
import pickle
import time

IMAGE_SIZE=424

NUM_EXAMPLES_PER_EPOCH_FOR_TRAINING=55000
NUM_EXAMPLES_PER_EPOCH_FOR_VALIDATION=6578
batch_size=16
training_iters=5000

logs_path='/tmp/tensorflow/Galaxy_Classification'
train_dir='/tmp/Galaxy_Classification_train'
sess=tf.InteractiveSession()
    
def read_images(filename_queue):
    class GalaxyZooRecord(object):
        pass
    result=GalaxyZooRecord()
    
    result.height=424
    result.width=424
    result.depth=3
    image_bytes=result.height*result.width*result.depth
    
    reader=tf.WholeFileReader()
    result.key,value=reader.read(filename_queue)
    image_bytes=tf.decode_raw(value,tf.uint8)
    
    depth_major=tf.reshape(image_bytes,[result.depth,result.height,result.width])
    result.uint8image=tf.transpose(depth_major,[1,2,0])
    
    return result

def read_labels(filename_queue):
    reader=tf.TextLineReader()
    key,value=reader.read(filename_queue)
    record_defaults = [[0.0] for _ in range(37)]
    label=tf.stack(tf.decode_csv(value,record_defaults=record_defaults))
    return label
    
def generate_training_data(batch_size):
    images_filenames=[os.path.join(root,name) for root,dirs,files in os.walk('images_training') for name in files]
    images_filename_queue=tf.train.string_input_producer(images_filenames)
    
    read_input=read_images(images_filename_queue)
    reshaped_image=tf.cast(read_input.uint8image,tf.float32)
    height=IMAGE_SIZE
    width=IMAGE_SIZE
    
    cropped_image=tf.image.central_crop(reshaped_image,0.5)
    resized_image=tf.image.resize_images(cropped_image,[90,90])
    
    rotated_image=tf.contrib.image.rotate(resized_image,random.uniform(0,math.pi))
    
    distorted_image=tf.image.random_flip_left_right(rotated_image)
    distorted_image=tf.image.random_brightness(distorted_image, max_delta=63)
    distorted_image=tf.image.random_contrast(distorted_image, lower=0.2, upper=1.8)
    distorted_image=tf.image.central_crop(distorted_image,0.5)
   
    float_image = tf.image.per_image_standardization(distorted_image)
    float_image.set_shape([45, 45, 3])

    labels_filename_queue=tf.train.string_input_producer(["training_solutions.csv"],shuffle=False)
    label=read_labels(labels_filename_queue)
    coord=tf.train.Coordinator()
    threads=tf.train.start_queue_runners(coord=coord)
    
    min_queue_examples=int(0.5*NUM_EXAMPLES_PER_EPOCH_FOR_TRAINING)
    print ('Filling queue with %d GalaxyZoo examples before starting to train. ' 'This will take a few minutes.' % min_queue_examples)
    
    return _generate_image_and_labels_batch(float_image,label,min_queue_examples,batch_size)

def _generate_image_and_labels_batch(image,label,min_queue_examples,batch_size):
    num_preprocess_threads = 16
    images,labels=tf.train.batch([image,label],batch_size=batch_size,num_threads=num_preprocess_threads,capacity=min_queue_examples+3*batch_size)
    # Display the training images in the visualizer.
    tf.summary.image('images', images)

    return images,labels
    
images,labels=generate_training_data(batch_size=batch_size)

# BUILDING THE CONVOLUTIONAL NETWORK

In [ ]:
def activation_summaries(x):
    #Creates a summary that provides a histogram of activations
    #Creates a summary that measures the sparsity of activations
    tf.summary.histogram('Activations',x)
    tf.summary.scalar('Sparsity',tf.nn.zero_fraction(x)) # Returns faction of zeros in value


def ConVNet(images):
    #Conv1
    with tf.variable_scope('conv1') as scope:
        kernel=tf.get_variable('weights', shape=[6,6,3,32],dtype=tf.float32,initializer=tf.truncated_normal_initializer(stddev=0.01,dtype=tf.float32),regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0)))
        conv=tf.nn.conv2d(images,kernel,[1,1,1,1],padding='VALID')
        biases=tf.get_variable('biases',[32],initializer=tf.constant_initializer(0.1))
        conv1=tf.nn.relu(tf.nn.bias_add(conv,biases),name=scope.name)
        activation_summaries(conv1)
        
    
    #Conv2
    with tf.variable_scope('conv2') as scope:
        kernel=tf.get_variable('weights', shape=[5,5,32,64],dtype=tf.float32,initializer=tf.truncated_normal_initializer(stddev=0.01,dtype=tf.float32),regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.01)))
        conv=tf.nn.conv2d(conv1,kernel,[1,1,1,1],padding='VALID')
        biases=tf.get_variable('biases',[64],initializer=tf.constant_initializer(0.1))
        conv2=tf.nn.relu(tf.nn.bias_add(conv,biases),name=scope.name)
        activation_summaries(conv2)
    #pool2
    pool2=tf.nn.max_pool(conv2,ksize=[1,20,20,1],strides=[1,1,1,1],padding='VALID',name='pool2')
    
    
    #Conv3
    with tf.variable_scope('conv3') as scope:
        kernel=tf.get_variable('weights', shape=[3,3,64,128],dtype=tf.float32,initializer=tf.truncated_normal_initializer(stddev=0.01,dtype=tf.float32),regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.01)))
        conv=tf.nn.conv2d(pool2,kernel,[1,1,1,1],padding='VALID')
        biases=tf.get_variable('biases',[128],initializer=tf.constant_initializer(0.1))
        conv3=tf.nn.relu(tf.nn.bias_add(conv,biases),name=scope.name)
        activation_summaries(conv3)
    #pool3
    pool3=tf.nn.max_pool(conv3,ksize=[1,8,8,1],strides=[1,1,1,1],padding='VALID',name='pool3')
    
    #Conv4
    with tf.variable_scope('conv4') as scope:
        kernel=tf.get_variable('weights', shape=[3,3,128,128],dtype=tf.float32,initializer=tf.truncated_normal_initializer(stddev=0.01,dtype=tf.float32),regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.01)))
        conv=tf.nn.conv2d(pool3,kernel,[1,1,1,1],padding='VALID')
        biases=tf.get_variable('biases',[128],initializer=tf.constant_initializer(0.1))
        conv4=tf.nn.relu(tf.nn.bias_add(conv,biases),name=scope.name)
        activation_summaries(conv4)
    #pool4
    pool4=tf.nn.max_pool(conv4,ksize=[1,2,2,1],strides=[1,1,1,1],padding='VALID',name='pool4')
    pool4_flat=layers.flatten(pool4)
    
    
    #FC1
    fc1=layers.fully_connected(pool4_flat,num_outputs=2048,activation_fn=tf.nn.relu,
                        normalizer_fn=layers.batch_norm,
                        weights_initializer=tf.truncated_normal_initializer(stddev=0.001,dtype=tf.float32),
                        biases_initializer=tf.constant_initializer(0.01))
    fc1=layers.dropout(fc1,keep_prob=0.7)
    
    
    #FC2
    fc2=layers.fully_connected(fc1,num_outputs=2048,activation_fn=tf.nn.relu,
                               weights_initializer=tf.truncated_normal_initializer(stddev=0.001,dtype=tf.float32),
                               normalizer_fn=layers.batch_norm,
                               biases_initializer=tf.constant_initializer(0.01))
    fc2=layers.dropout(fc2,keep_prob=0.7)
    
    #Softmax Layer
    softmax_linear=layers.fully_connected(fc2,num_outputs=37,activation_fn=tf.nn.softmax,
                                weights_initializer=tf.truncated_normal_initializer(stddev=0.01,dtype=tf.float32),
                                biases_initializer=tf.constant_initializer(0.1))
    activation_summaries(softmax_linear)
    
    return softmax_linear

logits=ConVNet(images)
cost=tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(labels,logits))))

optimizer=tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)
init = tf.global_variables_initializer()

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter(logs_path)
saver = tf.train.Saver(tf.global_variables())
    
def train():
    with tf.Session() as sess:
        sess.run(init)
        step=0
        loss_total=0
        writer.add_graph(sess.graph)
        
        while(step<training_iters):
            summary,_loss=sess.run([merged,optimizer,cost])
            writer.add_summary(summary, step)
            #Stores total loss in 100 iters 
            loss_total += loss
            if(step%1==0):
                print("Iter= " + str(step) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total))
                loss_total=0
                saver.save(session,"./Saver/model.ckpt")
                print("Model saved ")
            step+=1
    

In [ ]:
train()